In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Python MapReduce
<!-- requirement: small_data/B.csv -->
<!-- requirement: small_data/gutenberg -->
<!-- requirement: projects/mrjob -->
<!-- requirement: small_data/A.csv -->
<!-- requirement: small_data/users.csv -->
<!-- requirement: small_data/sales.csv -->


Yesterday we saw an example of using the Hadoop Streaming API in Python to implement a word count. Today we'll be taking a closer look at both Python MapReduce and how to run "real" jobs using an external cloud-based service. We'll also be taking a look at another implementation of word count in Python using a library called `mrjob`.


## Mrjob: a wrapper around Hadoop Streaming

`mrjob` has a few advantages over directly using Hadoop Streaming.
1. It is an open-source Python framework that provides a *pythonic API* to Hadoop Streaming and is actively developed by Yelp.
2. It more closely resembles the Java MapReduce paradigm that is the most commonly used in industry.
3. Since Yelp operates entirely inside Amazon Web Services, `mrjob`’s integration with Amazon's Elastic MapReduce (EMR) is very smooth and easy (using the `boto` package). We are partially using `mrjob` because it enables seamless resource-sharing on EMR, e.g. for the `mr.py` miniproject.

Some features:
- Run jobs on EMR, your own Hadoop cluster, or locally (for testing).
- Write multi-step jobs (one MapReduce step feeds into the next).

However, keep in mind that `mrjob` *will* be slower than using the straight-up Streaming API. 


### Example

Look at the script [`projects/mrjob/src/wordcount.py`](projects/mrjob/src/wordcount.py) in the `datacourse\mapreduce\` folder.  

1. The code consists mostly of a class that inherits from another class called `MRJob`:
```python
   class MRWordCount(MRJob):
```

1. A mapper which uses regular expressions to break up each line into words and yields examples of each:
```python
    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)
```

1. A reducer that sums up all instances of the words:
```python
    def reducer(self, word, counts):
        yield (word, sum(counts))
```

1. So far, we have only defined a MapReduce class `MRWordCount`.  Because we want this to be a script, we need to actually run it:

```python
    if __name__ == '__main__':
        MRWordCount.run()
```


## Python iterators and generator functions

Generators are functions that create iterators, for example with Python’s `yield` statement. They have the advantage that an element of a sequence is not produced until you actually need it. This can help a lot in terms of computational expensiveness or memory consumption depending on the task at hand.  This has two practical consequences in `MRJob`:
1. The keyword `yield` is like `return` but you can `yield` as any times as you want!  This allows you to output zero, one, or multiple records in the mapper. (Arguably, "MapReduce" is more accurately called "Flat MapReduce", but that seems less catchy.)
1. The values in a reduce (`counts` in our above example) are Python iterators so you can only run through them once.  How would you compute the mean of `counts`?



### Bottlenecks in MapReduce

The two most expensive things in a MapReduce are
1. **Network IO:** sending data back and forth between nodes, and 
1. **Disk IO:** persisting data to disk.


### Combiner

You might decide that this MapReduce is inefficient.  If the word `bear` appears twice in a line or node, it writes `(bear,1)` twice instead of `(bear,2)` once.  Doing the latter would reduce the network IO costs at the expense of Disk IO.  Fortunately, there is the notion of a `combiner`: for certain types of reducers, it [effectively runs reduce locally on the node](http://mrjob.readthedocs.org/en/latest/guides/concepts.html), potentially reducing network traffic.  [Here's some documentation on how to add a combiner to `mrjob`](https://pythonhosted.org/mrjob/job.html#mrjob.job.MRJob.combiner).  

**Exercise:**
Add a `combiner` to word count.

**Question:** What is a (possible) downside of a `combiner`?

### Warning:

A class that extends `MRJob` is not a normal class.  Usually in classes, we can store values and use them again later.  
```python
class Foo(object):
    def store(self, value):
        self._value = value
        
    def retrieve(self):
        return self._value
        
foo = Foo()
foo.store(3)
print(foo.retrieve())  # prints 3
```

Don't count on this with `MRJob`.  This class is more like DNA: every mapper and reducer gets the entire MapReduce, but the different nodes only run the portion of the code that are relevant for them.  For example, a mapper class will run `mapper` but not `reducer` and vice versa.  The only exception is that:
1. Each mapper node will run `mapper_init`, `mapper`, and `mapper_final`.
1. Each combiner node will run `combiner_init`, `combiner`, and `combiner_final`.  (That is, if the `combiner` is run...)
1. Each reducer node will run `reducer_init`, `reducer`, and `reducer_final`.

You _can_ define a variable in, say, `mapper_init` and use that variable in `mapper` and/or `mapper_final`.  Note that this means you still cannot share data between mapper nodes during the map phase, etc ...


### Running code:

To invoke `MRJob`, run
```bash
python wordcount.py input_file.txt | more
```

to count the words in `input_file.txt`.  To run the word count, on our project Gutenberg text, run (from `datacourse`):
```bash
python projects/mrjob/src/wordcount.py small_data/gutenberg/pg20417.txt | more
```

Note: if you want to use another version of Python (say Python 2 when the default is Python 3 or vice-versa), just use that command instead of `python`.  So to run explicitly in Python 3:
```bash
python3 projects/mrjob/src/wordcount.py small_data/gutenberg/pg20417.txt | more
```

**Exercises**:
1. Create a script called `src/unixwordcount.py`.  Make this script output the number of characters, words, and lines in a string of text, i.e. to mimic the <a href="http://en.wikipedia.org/wiki/Wc_(Unix)">Unix word count program</a>.
1. Use MR to compute the total amount of sales per user in `data/sales.tsv`.


### Choosing the right runner


The execution of MapReduce jobs depends on the [runner](https://pythonhosted.org/mrjob/guides/runners.html) you choose. The local runners are useful for testing and development, but most big jobs will be run on cluster, possibly in the cloud.  The options are:

**`-r inline`** Runs the MapReduce job serially, one job at a time.  Good for testing small jobs, but you don't want to do real jobs this way.  This is the default runner.

**`-r local`** Simulates a Hadoop cluster with UNIX processes.  This lets you test that your tasks will work in parallel, but there is no resource management other than that provided by the operating system.  There is a danger of memory exhaustion if too many processes request too many resources.

**`-r hadoop`** Run on the local Hadoop cluster.  Input data can either be on the local file system or HDFS (specified as `hdfs:///user/jovyan/...`).  As of this writing, a bug in `mrjob` prevents local directories being used if they are specified with a trailing slash.  Hadoop manages the cluster's resources, so there should be no danger of memory exhaustion.

**`-r emr` or `-r dataproc`** Run on Amazon's Elastic MapReduce or Google's Dataproc platform, assuming these are configured.  See the [Cloud notebook](MR_Mrjob_in_the_Cloud.ipynb) for details.


### Keeping track of resources


You can use the web UI at port 8042 to keep track of Hadoop MapReduce jobs and monitor their progress.  The `hadoop job` command offers a command line tool for this.  The `-list` and `-kill` options will probably be useful.  When running distributed jobs locally you also need to be careful of things like zombie tasks or temporary files sticking around. Use commands like `top` or `df` to keep track of your memory and disk usage. You may need to occasionally manually kill stray tasks.

When a MapReduce step fails, `mrjob` attempts to display the traceback that caused the problem.  This works well with the inline and local runners, but a permissions problem currently breaks it with the Hadoop runner.  You can look at the logs by hand, in `/var/log/hadoop-yarn/userlogs/application*/`.  These are owned by the yarn user and are, oddly enough, not world-readable, so you will want to switch users before examining them (`sudo -su yarn`).  You will need to search for output with error messages, for example by grepping for "Traceback".

## Computing statistics


Suppose we want to take the mean grouped by a key.  Remember that the `values` in `reducer` are an *iterator*, not a list.  This has a practical consequence:

**Anti-example:** What is wrong with this line of code?
```python
    def reducer(self, key, values):
       yield key, 1. * sum(values) / len(values)
```

**Questions:**
1. Write a MapReduce to compute the mean and variance of `data/numbers.txt` without storing the entire sequence.  How would you add a `combiner`?
1. How would you compute the (approximate) median, 25% quantile, or 75% quantile?
1. How would you normalize features by (e.g.) converting counts to percentages?  That is, if you have a list of numbers $N_i$, you want a list of numbers,
    $$ \frac{N_i}{\sum_j N_j} .$$
    Can you do this in a single MapReduce?  *Hints:*   You need to compute the total $\sum_j N_j$ in the `mapper` (or, more appropriately, `mapper_final`).
1. How would we normalize features before submitting them to a machine learning algorithm?  That is, given features $X_{ij}$ in the above format, we want a MapReduce which outputs $X'_{ij}$ where
    $$ X'_{ij} = \frac{X_{ij} - \mu_j}{\sigma_j} $$
    where $\mu_j$ and $\sigma_j$ are the mean and standard deviation of column $j$ of $X_{ij}$.  Can you do this in a single MapReduce?
    

## Tips for writing MapReduce applications


The [documentation](https://pythonhosted.org/mrjob/guides/writing-mrjobs.html) for creating applications is quite thorough. There are a couple of details worth mentioning here:

* Setting `SORT_VALUES = True` in a mapper class will enable secondary sort, so that the values go to the reducer in lexographic order.
* Class inheritance works normally if you want to abstract away parts of your code.
* You can use [`jobconf`](https://pythonhosted.org/mrjob/guides/configs-hadoopy-runners.html#option-jobconf) to pass command-line arguments into your code. For example, if on the command line you enter `--jobconf "key.x=42"`, you can do:
``` python
from mrjob.compat import jobconf_from_env
x = int(jobconf_from_env("key.x"))
assert x == 42
```

## Translating from SQL


1. How would you filter by row in MapReduce?  In other words, how would you do this type of SQL query in MapReduce?
```SQL
SELECT * FROM table WHERE country = 'US';
```

1. How would you filter by column?  Why would you do this in MapReduce?
```SQL
SELECT col1, col2 FROM table;
```

1. How would you get a list of (distinct) users from your table?
```SQL
SELECT DISTINCT user FROM table;
```

1. Let's say you're trying to sum all sales by country in a MapReduce.  How would you write a "group by" MapReduce?  The equivalent SQL code would be:
``` SQL
SELECT sum(sales) FROM table GROUP BY country;
```

1. **A "hot" node in MapReduce:** Let's say there is a key is 'hot' and records with that key account for 90% of the data in that phase.  (For example, you are running total sales per country but the vast majority of your sales occur in the US.)  Since all the data with the same key gets sent to a single reducer, you have not taken advantage of the parallelization in MR.  How would you solve this?

## Joining data in MapReduce


Joining data in MapReduce typically involves using multiple tasks for a MapReduce.  For example, let's say we want to to compute total sales by country.  Roughly speaking, we are trying to do the equivalent of this in SQL:
```SQL
SELECT sum(sales.amount)
    FROM sales JOIN users
    ON sales.user = users.user
    GROUP BY users.country;
```

We will be joining `small_data/sales.tsv` and `small_data/users.tsv` to compute total sales by country. Notice that `MRJob` can take any number of arguments and all their contents get fed into mapper.  In the mapper, you will need to use the values in the first column to determine which table you are reading.  The full file is in [`projects/mrjob/src/join.py`](projects/mrjob/src/join.py)

1.  The initial mapper emits `userid, amount` (key, value) pairs for the sales records and `userid, country` (key,value) pairs for user records.  Since both sets of (key, value) pairs are placed on the same stream, we need a dummy string (`"amount"` and `"country"`) statement to denote which type of key pair this is:
```python
    def mapper1(self, _, line):
        row = line.split(",")
        if row[0] == "sales":
            yield row[3], ("amount", row[4])
        elif row[0] == "users":
            yield row[1], ("country", row[-1])
```
    
1.  The initial reducer aggregates the amounts by user.  Since we ultimately want the results by country, we emit that as the key.
```python
    def reducer1(self, userid, values):
        total = 0
        for type_, val in values:
            if type_ == "amount":
                total += int(val)
            elif type_ == "country":
                country = val
        yield country, total
```
    
1.  We need to run a second MapReduce which aggregates by country.  Since the results are already keyed by country, the second mapper does not need to do anything.  The second reducer is very simple:
```python
    def reducer2(self, country, totals):
        yield country, sum(totals)
```
    
1.  MR jobs are written to be composable: that is, we can take the output of one MR job and input it to another.  In `MRJob`, this is done by overriding the `steps` method to return a list of the individual `MRJob` classes which call mappers and reducers defined above in the class.  Check out the [`MRJob` documentation](https://pythonhosted.org/mrjob/guides/writing-mrjobs.html#multi-step-jobs) for more details.  
```python
    def steps(self):
        return [
            MRStep(mapper=self.mapper1, reducer=self.reducer1),
            MRStep(reducer=self.reducer2)  # identity mapper implied
        ]
```
    

## Counting in MapReduce


We often want to count things up (e.g., count how many records we processed).  It's tempting to do something like this:

**Anti-example:** What is wrong with this?
```python        
    def mapper(self, k, v):
        print((k, v), file=sys.stderr)
        yield f(k, v)
```

**Anti-example:** Why does this not really work well on a distributed system?
```python
    def mapper_init(self):
        self.counter = 0
        
    def mapper(self, k, v):
        self.counter += 1
        yield f(k, v)
        
    def mapper_final(self):
        print(self.counter, file=sys.stderr)
```

**Example**: Instead of the above, here's a mechanism provided to do counting:
```python
    def mapper(self, k, v):
        self.increment_counter("group name", "counter name", amount=1)
        yield f(k, v)
```

These counters are incremented at the end of the computation.

**Questions:**
1. What if you wanted to add a float (and only needed an approximate answer)?
1. What if you wanted to compute the mean?

### Another Multi-Step MapReduce


Another classic example of an MapReduce that requires two steps is squaring a matrix $M$.  The two steps are:
1. The first MapReduce emits $M_{ij}$ keyed by both $i$s and $j$'s (map) and then performing the multiplication $M_{ij} * M_{jk}$ (reduce) keyed by $(i,k)$.
2. The second MapReduce sums the values of $M_{ij} * M_{jk}$ along $j$ (reduce: it has an identity mapper).

**Question:** How much memory does normal matrix multiplication take?  How much memory does MapReduce matrix multiplication take?

**Exercises:**

1.  Write a MapReduce to multiply two matrices $A_{ij}$ and $B_{ij}$.  Assume that the elements of $A_{ij}$ are encoded sparsely in a `csv` format:

    `"A", i, j, v`
    
    That is, `i` and `j` are the indices and `v` is the value at position `i` and `j`.  $B_{ij}$ is similar.  See `small_data/A.csv` and `small_data/B.csv` for example inputs.  Note that the first column is there to distinguish between the two matrices because `mrjob` only takes one input file.  Hint: this requires two MapReduces chained together.
1.  Compute the set of all common friends between two individuals via a MapReduce.  Assume that the friendship matrix is encoded sparsely in a `csv` format:

    `u1, u2`
        
    The above row implies, `u1` is friends with `u2` (and this is a symmetric relationship).  The output should be of the form
    
    `u1, u2, 24`
    
    which would represent that there are 24 common friends shared by `u1` and `u2`.  Can you reuse your solution to the previous MapReduce to help?
1.  Write a MapReduce implementation of Naive Bayes.  The solution is what is called an *online* as opposed to *offline* algorithm.  Assume the input data are rows of a feature matrix $X_{ij}$ given in a `csv` format

    `f1, f2, f3, ..., fp`
    
    where `f1`, ... `fp` are floats that represent the $p$ features of a row in $X$.  The output should be one row
    
    `b1, b2, b3, ..., bp`
    
    where `b1`, ..., `bp` are the components of $\beta$, the coefficients of the model.  When would this be useful?
1.  Write linear or logistic regression as a MapReduce with the same input and output.
1.  What other machine learning algorithms can you write as a MapReduce?

### Exit Tickets

1. Explain to a layman why reducer functions must be associative. Must they be commutative as well? What about combiner functions?
1. Write a generator function that accepts as input a stream of all positive integers and yields only those divisible by $k$. 
1. Write a function `sum()` that accepts as input a stream of integers and returns the sum.

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*